In [4]:
import gradio as gr
import joblib
import json
import requests
from datetime import datetime

# Get the time now
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
# Get if we are in the morning or afternoon, evening or night
if int(current_time.split(":")[0]) < 12 and int(current_time.split(":")[0]) > 6:
    time_of_day = "morning"
elif int(current_time.split(":")[0]) < 18:
    time_of_day = "afternoon"
elif int(current_time.split(":")[0]) < 22:
    time_of_day = "evening"
else:
    time_of_day = "night"

# Load credentials.json
with open("../credentials.json") as f:
    credentials = json.load(f)

api_key = credentials["OpenWeatherMap"]["api_key"]


In [16]:
def get_weather_forecast(appid, lat, lon, units="metric", lang="en"):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    
    # Construct the API call
    call = f"{base_url}?appid={appid}&lat={lat}&lon={lon}&units={units}&lang={lang}"

    # Make the API call
    response = requests.get(call)

    if response.status_code == 200:
        data = response.json()
        weather = data["weather"][0]["description"]
        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        pressure = data["main"]["pressure"]
        wind_speed = data["wind"]["speed"]
        wind_direction = data["wind"]["deg"]
        rain = data["rain"]["1h"] if "rain" in data else 0
        
        return {
            "weather": weather,
            "temperature": temperature,
            "humidity": humidity, 
            "pressure": pressure,
            "wind_speed": wind_speed,
            "wind_direction": wind_direction,
            "rain": rain
        }
    else:
        print("Error calling API: status code: {}".format(response.status_code))
        return None

lat, lon = 55.676098, 12.568337

# Get the weather forecast for the specified city
forecast = get_weather_forecast(lat, lon, api_key, units="metric", lang="en")

if forecast:
    print("Import successful")
else:
    print("Import failed")

Error calling API: status code: 401
Import failed


In [ ]:
# Load the pre-trained model
model = joblib.load("../results/models/present-day_bc_ml_best_model.sav")

# Function to make predictions using the loaded model
def predict_output(out_temp, out_rh, pressure, wind_dir, wind_speed, rain, floor_area, building_height, orientation, construction_type, ceiling_insulation, number_people, day_period, history_temperature):
    # Process the inputs and make predictions
    # Replace this code with your actual model prediction logic
    output = model.predict([[out_temp, out_rh, pressure, wind_dir, wind_speed, rain, floor_area, building_height, orientation, construction_type, ceiling_insulation, number_people, day_period, history_temperature]])[0]
    return output

# Create the input interface
out_temp = gr.inputs.Slider(minimum=20, maximum=50, default=forecast["temperature"], label="Outside temperature (°C)")
out_rh = gr.inputs.Slider(minimum=0, maximum=100, default=forecast["humidity"], label="Outside relative humidity (%)")
pressure = gr.inputs.Slider(minimum=700, maximum=1100, default=forecast["pressure"], label="Pressure (hPa)")
wind_dir = gr.inputs.Slider(minimum=0, maximum=360, default=forecast["wind_direction"], label="Wind direction (°)")
wind_speed = gr.inputs.Slider(minimum=0, maximum=20, default=forecast["wind_speed"], label="Wind speed (m/s)")
rain = gr.inputs.Slider(minimum=0, maximum=10, default=forecast["rain"], label="Rain (mm/h)")
floor_area = gr.inputs.Slider(minimum=0, maximum=80, default=25, label="Floor area (m²)")
building_height = gr.inputs.Slider(minimum=0, maximum=4, default=2, label="Building height (m)")
orientation = gr.inputs.Slider(minimum=0, maximum=360, default=0, label="Orientation (°)")
construction_type = gr.inputs.Dropdown(["Light", ""], default="Light", label="Construction type")



# Create the output interface
output = gr.outputs.Text(label="Output")

# Define the app
gr.Interface(fn=predict_output, inputs=[], outputs=output, title="ML Model App").launch()

In [1]:
from toolbox.machine_learning import get_features_sets

print(get_features_sets("Aggregated"))

['Outdoor Dry Bulb Temperature', 'Outdoor Relative Humidity', 'Atmospheric Station Pressure', 'Wind Direction', 'Wind Speed', 'Precipitable Water', 'floor_area', 'volume', 'orientation', 'construction_type', 'ceiling_insulation', 'number_people', 'day_period', 'history_temperature']
